In [ ]:
!pip install gensim
!pip install tensorflow
!pip install tf-keras
!pip install datasets
import argparse
import json
import string
import random
import re
import os
import string
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import PorterStemmer
from gensim.models.word2vec import Word2Vec
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import pipeline
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
from google.colab import drive


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 26.7/26.7 MB 205.5 MB/s eta 0:00:01

In [ ]:
# !pip install torch==2.7.0 'torch_xla[tpu]==2.7.0'

In [ ]:
# !pip list | grep torch

In [ ]:
# !add-apt-repository ppa:graphics-drivers/ppa
# !apt update
# !apt install nvidia-384 nvidia-384-dev
# !apt-get install g++ freeglut3-dev build-essential libx11-dev libxmu-dev libxi-dev libglu1-mesa libglu1-mesa-dev
# !pip install torch torchvision torchaudio torch_xla -f https://download.pytorch.org/whl/torch_stable.html
# !wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1710-9-2-local_9.2.88-1_amd64
# !wget https://developer.nvidia.com/compute/cuda/9.2/Prod/patches/1/cuda-repo-ubuntu1710-9-2-local-cublas-update-1_1.0-1_amd64
# !dpkg -i cuda-repo-ubuntu1710-9-2-local_9.2.88-1_amd64
# !apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
# !apt-get update
# !apt-get install cuda
# !dpkg -i cuda-repo-ubuntu1710-9-2-local-cublas-update-1_1.0-1_amd64
# !apt-get install nvidia-cuda-toolkit
# !pip install torch torchvision torchaudio
# import torch

In [ ]:
# # !pip3 uninstall torch torchvision torchaudio
# # !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
# import torch
# print(torch.cuda.is_available())

In [ ]:
drive.mount('/content/drive')
# Specifying the File Path
train_data_dir = '/content/drive/My Drive/641/train.jsonl'
val_data_dir = '/content/drive/My Drive/641/val.jsonl'
test_data_dir = '/content/drive/My Drive/641/test.jsonl'
output_dir = '/content/drive/My Drive/641/spoiler_result.jsonl'

In [ ]:
def load_preprocess_data(file_path, test_flag):
    with open(file_path, 'r', encoding="utf8") as inp:
        # Read input data
        data = {
            'id': [],
            'title': [],
            'text': [],
            'label': [],
            'target': []
        }

        # extract data from jsonl file
        for i in inp:
            i = json.loads(i)
            if test_flag:
                data['id'].append(i['id'])
            else:
                data['id'].append(''.join(i['postId']))
            data['title'].append(''.join(i['postText']))
            data['text'].append(''.join(i['targetParagraphs']))

            if not test_flag:
                data['label'].append(''.join(i['tags']))
                data['target'].append(''.join(i['spoiler']))

        # assign corresponding value to input, output & id
            # clean text
        texts = [preprocess_text(text) for text in data['text']]
        titles = [preprocess_text(title) for title in data['title']]
        if not test_flag:
          spoilers = [preprocess_text(target) for target in data['target']]


        if debug:
            print(f"Sample spoiler: {spoilers[0:5]}")

        if not test_flag:
          data_dict = {
              "input_text" : texts,
              "input_title" : titles,
              "target_text" : spoilers
          }
        else: # for test set
          data_dict = {
              "input_text" : texts,
              "input_title" : titles,
          }

        dataset = Dataset.from_dict(data_dict)

    return dataset

In [ ]:
def preprocess_text(text):

    # # Convert to lowercase
    # text = text.lower()
    # # # Remove special characters (keep only letters, numbers, spaces & some special punctuations)
    # text = re.sub(r'[^a-zA-Z0-9\s,\"\?]', '', text)
    # Remove extra whitespace
    text = ' '.join(text.split())

    if stopword_removal:
        # remove stop words
        tokenised = tokenize(text)
        sw = load_stopword()
        cleaned = remove_stopwords(tokenised, sw)
        text = " ".join(token for token in cleaned)

    return text
def tokenize(text):
    """Tokenize the input text by removing punctuation and splitting into words."""

    tokenized_text = []

    for word in re.findall(r'\w+', text):
        tokenized_text.append(word)

    return tokenized_text


def load_stopword():
    res = []
    if stopword_ref == 'nltk':
        res = stopwords.words('english')
    else:
        stopword_set = []
        with open('../stopwords.txt', 'r', encoding='utf-8') as f:
            for line in f:
                stopword_set.append(line.strip())

        for item in stopword_set:
            for word in re.findall(r'\w+', item):
                # Key: \w matches any alphanumeric character and underscore
                res.append(word)
        res = set(res)
        # if debug:
        #     print(res)
    return res


def remove_stopwords(tokens, stopwords):

    # reduce word to simplest root form
    porter = PorterStemmer()
    cleaned_tokens = [porter.stem(word) for word in tokens if word.lower() not in stopwords]

    return cleaned_tokens


def save_model(model, filename):
    """Save the trained model to a file."""
    with open(filename, 'wb') as f:
        pickle.dump(model, f)
    if debug:
        print(f"Model saved to {filename}")




In [ ]:
# for testing google pegasus
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
# model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

In [ ]:
debug = True
retrain = True
stopword_removal = False  # True, False
stopword_ref = 'nltk'  # nltk, pre
ref_x = 'text'  # title, text

nltk.download('stopwords')

model_name = 't5-base'  # t5-small, t5-base, t5-large
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

LEARNING_RATE = 1e-2
TRAIN_BATCH_SIZE = 16
VAL_BATCH_SIZE = 16
NUM_EPOCHS = 2
WEIGHT_DECAY = 2
LOGGING_STEPS = 10

# LEARNING_RATE = 3e-4
# TRAIN_BATCH_SIZE = 8
# VAL_BATCH_SIZE = 8
# NUM_EPOCHS = 2
# WEIGHT_DECAY = 0.3
# LOGGING_STEPS = 10

# LEARNING_RATE = 8e-5
# TRAIN_BATCH_SIZE = 8
# VAL_BATCH_SIZE = 8
# NUM_EPOCHS = 5
# WEIGHT_DECAY = 0.1
# LOGGING_STEPS = 10
# question_prefix = "Question: what is the following inferring in passage:"

# LEARNING_RATE = 4e-5
# TRAIN_BATCH_SIZE = 8
# VAL_BATCH_SIZE = 8
# NUM_EPOCHS = 5
# WEIGHT_DECAY = 0.05
# LOGGING_STEPS = 10

# LEARNING_RATE = 2e-5
# TRAIN_BATCH_SIZE = 8
# VAL_BATCH_SIZE = 8
# NUM_EPOCHS = 3
# WEIGHT_DECAY = 0.01
# LOGGING_STEPS = 10

In [ ]:
# Question: Find in passage, what is '{title}' inferring in passage? Passage: {text}
# Question: What is the key message that '{title}' is inferring in passage? Passage: {text}
#
# Current Best
# Question: What is the key spoiler that '{title}' is inferring in passage? Passage: {text}
def preprocess_dataset(dataset):

    inputs = [f"Question: What is the key spoiler that '{title}' is inferring in passage? Passage: {text}"
              for title, text in zip(dataset['input_title'], dataset['input_text'])]
    targets = [spoiler for spoiler in dataset['target_text']]

    # prepare data for model
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')
    model_targets = tokenizer(targets, max_length=30, truncation=True, padding='max_length')
    model_inputs["labels"] = model_targets["input_ids"]

    return  model_inputs

In [ ]:
# clean text & return a dataset (w/ text, title, target)
train_dataset = load_preprocess_data(train_data_dir, test_flag=False)
val_dataset = load_preprocess_data(val_data_dir, test_flag=False)


# apply tokenization and preprocessing of dataset
tokenized_train = train_dataset.map(preprocess_dataset, batched = True)
tokenized_val = val_dataset.map(preprocess_dataset, batched = True)


if debug:
    print(f"Sample tokenized train input: {tokenized_train[0]['input_text']}")
    print(f"Sample tokenized train target: {tokenized_train[0]['target_text']}")

# =========================================================================
# training


training_args = TrainingArguments(
    output_dir= "results",
    eval_strategy="epoch",
    learning_rate= LEARNING_RATE,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size= VAL_BATCH_SIZE,
    num_train_epochs= NUM_EPOCHS,
    weight_decay= WEIGHT_DECAY,
    logging_dir= "logs",
    logging_steps= LOGGING_STEPS,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)

# Fine-tune the model
trainer.train()

# Evaluate the model on the validation dataset
eval_results = trainer.evaluate()

# Print the evaluation results
print(f"Evaluation results: {eval_results}")

# ==========================================================================

# testing
with open(test_data_dir, 'r', encoding="utf8") as inp, open(output_dir, 'w', encoding="utf8") as out:
    # Read input data
    data = {
        'id': [],
        'title': [],
        'text': [],
        'label': [],
        'target': []
    }

    # extract data from jsonl file
    for i in inp:
        i = json.loads(i)
        data['id'].append(i['id'])
        data['title'].append(''.join(i['postText']))
        data['text'].append(''.join(i['targetParagraphs']))

    texts = [preprocess_text(text) for text in data['text']]
    titles = [preprocess_text(title) for title in data['title']]
    id = [id for id in data['id']]

    # prepare inputs for the model
    inputs = [f"Question: What is the key spoiler that '{title}' is inferring in passage? Passage: {text}" for title, text in zip(titles, texts)]

    # Tokenize inputs
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')

    # Generate predictions
    predictions = model.generate(input_ids=torch.tensor(model_inputs['input_ids']).to('cuda'))

    # Decode predictions
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    count = 0
    # Write results to output file
    for i, pred in enumerate(decoded_preds):
        if count == 0:
          print(f'input sample: {inputs[0]}')
          print(f'prediction sample: {pred}')
          count = 1
        out.write(json.dumps({'id': id[i], 'spoiler': pred}) + '\n')

    # with open(output_dir, 'r') as f:
    #   for line in f:
    #       data = json.loads(line)
    #       predictions.append(data)

    #   # Create DataFrame
    #   df = pd.DataFrame(predictions)

    #   # Save as CSV
    #   df.to_csv('/content/drive/My Drive/641/task2_submission.csv', index=False)